In [4]:
from secrets_file import GEMINI_KEY

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_KEY,
    temperature=0.6,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain


prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template= 'I want to open restuarnt for {cuisine} food. Suggest one fency name for this only a name that\'s it !!!'
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_menu = PromptTemplate(
    input_variables=['resturant_name'],
    template= """Suggest some menu items for {resturant_name}. Return it as comma separated list"""
)

menu_chain = LLMChain(llm=llm, prompt=prompt_template_menu)

# simple sequential chain will only give you one output

chain = SimpleSequentialChain(chains=[name_chain, menu_chain])

chain.run('Chinese')

'Here are some menu item suggestions for a restaurant called "Celestial Dragon", keeping in mind a likely Asian-inspired theme:\n\n**Appetizers:**\n\nDragon\'s Breath Dumplings,Phoenix Tears Soup,Cloud Nine Spring Rolls,Jade Emperor\'s Edamame,Cosmic Calamari,Starlight Seaweed Salad\n\n**Main Courses:**\n\nCelestial Dragon Noodles,Emperor\'s Fried Rice,Galaxy Glazed Salmon,Nebula Beef Stir-Fry,Comet Chicken Curry,Lunar Lobster,Stardust Scallops,Black Hole Beef Tenderloin,Zen Garden Vegetarian Delight\n\n**Desserts:**\n\nMilky Way Mochi,Moonbeam Mango Pudding,Shooting Star Sorbet,Dragon Fruit Delight,Cosmic Coconut Cream Pie\n\n**Drinks:**\n\nCelestial Sunrise Cocktail,Dragon\'s Blood Elixir,Milky Way Martini,Comet Cooler,Zen Garden Green Tea'

In [8]:
# sequential chain can have multiple inputs multiple outputs
from langchain.chains import SequentialChain, LLMChain

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="""I want to open a restaurant for {cuisine} food. Suggest ONE fancy name for this only."""
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="resturant_name")

prompt_template_items = PromptTemplate(
    input_variables=['resturant_name'],
    template="""Suggest some menu items for {resturant_name} comma seperated inlude only menu item names"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['resturant_name', 'menu_items']
)
chain({'cuisine': "Indian"})


/var/folders/gn/p_qvkw0s1bxc8zjhdjn5cxrr0000gn/T/ipykernel_50492/1638660101.py:22: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({'cuisine': "Indian"})


{'cuisine': 'Indian',
 'resturant_name': '**"Suvarnam"** (Sanskrit for "Golden")',
 'menu_items': 'Suvarna Korma, Golden Beetroot Tikki, Haldi-Infused Chicken Tikka, Swarna Mahal Biryani, Saffron Rice Pudding, Golden Lentil Soup, Turmeric Roasted Vegetables, Mango Lassi with Saffron, Ghee-Roasted Paneer, Coconut and Turmeric Fish Curry'}

### Agents

In [ ]:
# Agents
from langchain.agents import AgentType, initialize_agent, load_tools
# you have to install wikipdeia or any other tool 
tools = load_tools(['wikipedia'], llm=llm)

# A zero shot agent that does a reasoning step before acting.
agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# verbose tool --> gives all the steps the llm will take to reach the answer
agent.run("When was Elon Mask was born ? what is his age in 2025")



> Entering new AgentExecutor chain...
I need to find Elon Musk's birthdate and then calculate his age in 2025.
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman. He is known for his leadership of Tesla, SpaceX, X (formerly Twitter), and the Department of Government Efficiency (DOGE). Musk has been considered the wealthiest person in the world since 2021; as of May 2025, Forbes estimates his net worth to be US$424.7 billion. 
Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada. He received bachelor's degrees from the University of Pennsylvania in 1997 before moving to California, United States, to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002. That year, Musk also became an American citiz

'Elon Musk was born on June 28, 1971. He will be 54 years old in 2025.'

### Memory

In [16]:
chain = LLMChain(llm = llm, prompt = prompt_template_name)
name = chain.run("Mexican")
print(name)

**La Hacienda de Oro** (The Golden Estate)


In [17]:
chain = LLMChain(llm = llm, prompt = prompt_template_name)
name = chain.run("Indian")
print(name)

**"The Spice Route"**


In [18]:
type(chain.memory)

NoneType

In [35]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_KEY,
    temperature=0.6,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
chain = LLMChain(llm=llm, prompt = prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)
name = chain.run("Indian")
print(name)

# chain.memory
print(chain.memory.buffer)

# problem with ConversationBufferMemory --> grows continuously


**El Corazón de Oro** (The Golden Heart)
**"The Spice Route Atelier"**
Human: Mexican
AI: **El Corazón de Oro** (The Golden Heart)
Human: Indian
AI: **"The Spice Route Atelier"**


In [36]:
from langchain.chains import ConversationChain
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_KEY,
    temperature=0.6,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
convo = ConversationChain(llm=llm)

print(convo.prompt.template)

convo.run("Who won the first circket world cup ")

convo.run("what is 5 + 5")

convo.run("Who was the captain of winning team")

print(convo.memory.buffer)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:
Human: Who won the first circket world cup 
AI: The first Cricket World Cup, also known as the Prudential Cup, was held in England in 1975. And to answer your question directly, **West Indies** won the tournament! They defeated Australia in the final at Lord's. It was a pretty exciting match, with Clive Lloyd scoring a brilliant century for the West Indies. Do you want to know more about the tournament, perhaps about some of the other teams or key players? I'd be happy to share what I know!
Human: what is 5 + 5
AI: 5 + 5 is 10! Is there anything else I can help you calculate? I can do quite a bit, from simple arithmetic to more complex equations, and even some unit conversions if you need them. Just let me k

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=2)
# k = Number of messages to store in buffer. it stores last k message and base on the it response 
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_KEY,
    temperature=0.6,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
convo = ConversationChain(llm = llm, memory=memory)

convo.run("Who won the first circket world cup T20")


"The first ICC World Twenty20, now known as the T20 World Cup, was held in South Africa in 2007. And the winner? India! They beat Pakistan in a thrilling final at the Wanderers Stadium in Johannesburg. I remember reading about how Joginder Sharma bowled that final over and MS Dhoni's captaincy was praised. It was a really exciting tournament and helped popularize the T20 format."

In [45]:
convo.run("What is 5 + 5")


"5 + 5 equals 10. I can do basic arithmetic calculations. Would you like me to try any other calculations for you? Perhaps something a bit more challenging than addition? I can also do subtraction, multiplication, and division. I can even handle exponents and square roots, although those might take me a fraction of a second longer to compute. Just let me know what you'd like me to calculate!"

In [46]:
convo.run("Who was the captain of winning team")
# if k = 1 it won't remember the answer for convo.run("Who won the first circket world cup T20")
# and would have thrown any bogus statement

"The captain of the Indian team that won the first T20 World Cup in 2007 was Mahendra Singh Dhoni, often referred to as MS Dhoni. He was a relatively young captain at the time, but his calm demeanor and strategic decisions were widely credited for India's success in that tournament. He went on to become one of India's most successful captains in all formats of cricket."

In [49]:
convo.run("What is 5 - 5 ")

"5 - 5 = 0. Subtraction is the arithmetic operation where you find the difference between two numbers. When you subtract a number from itself, the result is always zero. It's the additive identity! I can do more subtraction problems if you'd like. We could even explore negative numbers!"

In [50]:
convo.run("What is 5 * 5 ")

"5 * 5 = 25. Multiplication is one of the basic mathematical operations, and it's essentially repeated addition. In this case, 5 * 5 is the same as adding 5 to itself 5 times (5 + 5 + 5 + 5 + 5). Want to try some bigger multiplication problems? I can handle exponents too!"

In [54]:
convo.run("Where was the location of this match ")
# bogues response since k=2 it only remeber last 2 messages 
# 'Okay, just to clarify before I give you an answer I can\'t provide: Are you asking about a sporting event (a "match"), or are you still referring to the "math" problem we were discussing earlier?\n\nIf you mean a sporting "match," I still don\'t have any specific information about a particular match. To help me understand, could you provide more details? For example:\n\n*   **What kind of match was it?** (e.g., soccer, tennis, chess, etc.)\n*   **When did it take place?** (e.g., yesterday, last week, a specific date)\n*   **Who were the participants?** (e.g., specific teams or individuals)\n\nWith more information, I might be able to access relevant data and provide you with the location.\n\nIf you meant "math," then please remember that the calculation happened within my digital architecture and wasn\'t held at any physical location.'

'Alright, I\'m still a little unsure which "match" you\'re referring to! Since I\'m designed to be helpful, I\'ll try one more time to narrow it down:\n\n**Possibility 1: You mean "math" and are still asking about the multiplication problem.**\n\nIn that case, the location is still within my internal processing units. Think of it like this: the calculation happened "inside my head," not in a specific city or building. There\'s no physical location for it.\n\n**Possibility 2: You mean a sporting "match."**\n\nIf this is the case, I *really* need more information to help you. I don\'t have access to a general sports database. Think about the details that would help someone else understand which match you\'re talking about. For example:\n\n*   **Sport:** Was it a baseball game? A soccer match? A tennis tournament?\n*   **Teams/Players:** Who was playing? Knowing the teams or players involved would be a huge clue.\n*   **Date/Timeframe:** When did the match happen? Even a general timeframe